In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.utils import resample
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
from mpl_toolkits import mplot3d
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import os

In [ ]:
df_train = pd.read_excel("/home/rafael/Documentos/FACOM/Douturado/Doutorado/webcrawler/medicamentos.xlsx",index_col=0,dtype=str)
df_train.head()
list_preprossing=[]

In [ ]:
for dta in range(df_train.shape[0]):
    value=str(df_train.iloc[dta,1])
    # value=str(df_train.iloc[dta,0])+" "+str(df_train.iloc[dta,1]) +" "+str(df_train.iloc[dta,2])+" "+str(df_train.iloc[dta,3])
    list_preprossing.append(value)
list_preprossing=np.array(list_preprossing)
print(list_preprossing)

In [ ]:
# print(list_preprossing)
print(list_preprossing.shape)

In [ ]:
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens',device='cuda')

In [ ]:
corpus_embeddings = embedder.encode(list_preprossing,show_progress_bar=True)

In [ ]:
for data in corpus_embeddings:
    print(data.shape)

In [ ]:
tsne = TSNE(random_state = 42, n_components=2)
pca = PCA(n_components=2)
lda = LinearDiscriminantAnalysis(n_components=2)

In [ ]:
embeddings2d_pca = pca.fit_transform(corpus_embeddings)
# embeddings2d_lda = lda.fit_transform(corpus_embeddings,list_preprossing)


In [ ]:
knn_4 = KMeans(n_clusters=4,n_init=40)
modelo_knn_3 = knn_4.fit_predict(embeddings2d_pca)

In [ ]:
list_name =list(df_train.iloc[:,0])#lista com os nomes do medicamento
# print(list_name)
# print(modelo_knn_3)
classes=[] #dicionario para armazenar a classe. key(medicamento):value(classe)
color = {0:'.b',1:'.r',2:'.c',3:'.y',4:'xb',5:'xr',6:'xc',7:'xy',8:'xk',9:'xb',10:'xr',11:'xc',12:'xy',13:'xk'} #dicionario  de classe e cores
colors=[ color[x] for x in modelo_knn_3] # modelo_knn_3 retorna uma lista de valores [2,1,3,4]
fig,ax = plt.subplots(1,figsize=(10,10))
for i, cl_xy in enumerate(zip(colors,embeddings2d_pca)): #embeddings2d possui x e y
    cl=cl_xy[0]
    xy=cl_xy[1]
    ax.plot(xy[0],xy[1],cl)
    # print(df_train.iloc[i].name)
    classes.append({'nome':df_train.iloc[i].name,'ativo':list_name[i],"classes":df_train['classes'].iloc[i],"classe_pred":modelo_knn_3[i]})
    # classes.append({'nome':df_train.iloc[i].name,'ativo':list_name[i],"classes":df_train['classes'].iloc[i],"classe_pred":modelo_knn_3[i]})

# for i, cl_xy in enumerate(zip(colors,embeddings2d)): #embeddings2d possui x e y
#     cl=cl_xy[0]
#     xy=cl_xy[1]
#     ax.plot(xy[0],xy[1],cl)
#     print(df_train.iloc[i].name)
#     classes.append({'nome':df_train.iloc[i].name,'ativo':list_name[i],"classes":df_train['classes'].iloc[i],"classe_pred":modelo_knn_3[i]})
fig.savefig('my_plot.png')


In [ ]:
frame_result = pd.DataFrame(classes).sort_values("classe_pred",axis=0)

In [ ]:
docs_df = pd.DataFrame(list_preprossing,columns=["Doc"])
docs_df['Topico'] = modelo_knn_3
docs_df['Doc_ID'] = range(len(docs_df))
docs_per_topic=docs_df.groupby(["Topico"],as_index=False).agg({"Doc":' '.join})

In [ ]:

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(data))


In [ ]:

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topico'])
                   .Doc
                   .count()
                   .reset_index()
                   .rename({"Topico": "Topico", "Doc": "Size"}, axis='columns')
                   .sort_values("Size", ascending=False))
    return topic_sizes

top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)


In [ ]:
topic_sizes